In [ ]:
import instaloader
from datetime import datetime, timedelta
import time


loader = instaloader.Instaloader()


username = input("Enter your Instagram username: ")
try:
    loader.load_session_from_file(username)
    print("Session loaded successfully.")
except FileNotFoundError:
    print("No saved session found. Logging in...")
    password = input("Enter your Instagram password: ")
    loader.login(username, password)
    loader.save_session_to_file()
    print("Session saved successfully.")

profile = instaloader.Profile.from_username(loader.context, username)

followings_file = "followings.txt"
followers_file = "followers.txt"
non_followers_file = "non_followers.txt"
inactive_accounts_file = "inactive_accounts.txt"

def save_usernames(file_path, usernames):
    with open(file_path, "w") as file:
        file.writelines(f"{username}\n" for username in usernames)

print("Fetching followings...")
followings = []
for followee in profile.get_followees():
    followings.append(followee.username)
    if len(followings) % 100 == 0:  
        print(f"Fetched {len(followings)} followings so far...")
        save_usernames(followings_file, followings)
        time.sleep(10)  
save_usernames(followings_file, followings)
print(f"Fetched all {len(followings)} followings. Saved to {followings_file}.")


print("Fetching followers...")
followers = []
for follower in profile.get_followers():
    followers.append(follower.username)
    if len(followers) % 100 == 0:  
        print(f"Fetched {len(followers)} followers so far...")
        save_usernames(followers_file, followers)
        time.sleep(10)  
save_usernames(followers_file, followers)
print(f"Fetched all {len(followers)} followers. Saved to {followers_file}.")


print("Identifying accounts that don't follow you back...")
non_followers = [user for user in followings if user not in followers]
save_usernames(non_followers_file, non_followers)
print(f"Found {len(non_followers)} accounts that don't follow you back. Saved to {non_followers_file}.")

print("Checking last posts of non-followers...")
inactive_accounts = []
ten_days_ago = datetime.now() - timedelta(days=10)

for username in non_followers:
    with open(inactive_accounts_file, "w") as file:
        for username, last_post_date in inactive_accounts:
            if last_post_date:
                file.write(f"{username} - Last post: {last_post_date.strftime('%Y-%m-%d')}\n")
            else:
                file.write(f"{username} - No posts found.\n")
        print(f"Found {len(inactive_accounts)} inactive accounts. Saved to {inactive_accounts_file}.")
    try:
        account = instaloader.Profile.from_username(loader.context, username)
        posts = list(account.get_posts())
        if posts:
            last_post_date = posts[0].date
            if last_post_date < ten_days_ago:
                inactive_accounts.append((username, last_post_date))
        else:
            inactive_accounts.append((username, None))
            time.sleep(5)  
    except Exception as e:
        print(f"Error fetching posts for {username}: {e}")


with open(inactive_accounts_file, "w") as file:
    for username, last_post_date in inactive_accounts:
        if last_post_date:
            file.write(f"{username} - Last post: {last_post_date.strftime('%Y-%m-%d')}\n")
        else:
            file.write(f"{username} - No posts found.\n")
print(f"Found {len(inactive_accounts)} inactive accounts. Saved to {inactive_accounts_file}.")
